#### This notebook will be mainly used for the capstone project.

In [1]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


### Import necessary Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner
import requests # requests for getting the HTML contents of the website 
import lxml.html as lh # parsing the relevant fields
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

### Scrap Toronto Neighborhood Data

Download table from Wikipedia

In [106]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url, header=0)[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Eliminate cells with a borough that is Not assigned

In [107]:
df=df[~df.Borough.str.contains("Not assigned")]
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Combine by postcode, with neighborhoods separated with a comma

In [108]:
df=df.groupby('Postcode').agg([('Neighbourhood', ', '.join)])
df.columns="Borough","Neighbourhood"
df.head()

,Borough,Neighbourhood
Postcode,,
M1B,"Scarborough, Scarborough","Rouge, Malvern"
M1C,"Scarborough, Scarborough, Scarborough","Highland Creek, Rouge Hill, Port Union"
M1E,"Scarborough, Scarborough, Scarborough","Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


Eliminate duplicate entries for Borough

In [99]:
df['Borough']= df['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
df.head()

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [104]:
print(df.loc[df['Borough']=="Queen'sPark"])
df.loc[df['Neighbourhood']=="Not assigned",'Neighbourhood']=df.loc[df['Neighbourhood']=="Not assigned",'Borough']
print(df.loc[df['Borough']=="Queen'sPark"])

              Borough Neighbourhood
Postcode                           
M7A       Queen'sPark   Queen'sPark
              Borough Neighbourhood
Postcode                           
M7A       Queen'sPark   Queen'sPark


In [105]:
df.shape

(103, 2)